In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 9 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

19 December 2022 ; 16:24:27


In [3]:
features = ['PTS','VORP','FG','WS','MP','TRB','AST','PER','BPM']#, 'GS'] ### GS
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 8
# Propotion of maximum games played
gp = 1/4

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/roy/roy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('roy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,12,0,5.8,1.1,2.0,0.542,...,4.17,113.54,109.23,4.31,3.78,113.43,109.50,3.93,1,1
10,MarJon Beauchamp,SF,22,MIL,18,6,15.5,2.2,5.6,0.400,...,4.17,113.54,109.23,4.31,3.78,113.43,109.50,3.93,1,1
16,A.J. Lawson,SG,22,MIN,1,0,2.0,1.0,1.0,1.000,...,-0.77,113.10,113.83,-0.73,-1.55,112.85,114.37,-1.52,1,1
24,Josh Minott,SF,20,MIN,4,0,3.3,0.8,1.0,0.750,...,-0.77,113.10,113.83,-0.73,-1.55,112.85,114.37,-1.52,1,1
33,Wendell Moore Jr.,SG,21,MIN,13,2,6.6,0.8,1.8,0.417,...,-0.77,113.10,113.83,-0.73,-1.55,112.85,114.37,-1.52,1,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # Minimum games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP' ,'Tm', 'PER', 'VORP', 'WS', 'FG', 'PTS', 'Seed', 'Share']]
res.head(5)

,Player,MP,Tm,PER,VORP,WS,FG,PTS,Seed,Share
0,Paolo Banchero,34.8,ORL,17.9,1.708333,5.466667,7.1,22.0,12,0.628637
1,Bennedict Mathurin,29.0,IND,13.8,-0.529032,2.645161,5.6,17.9,9,0.219234
2,Walker Kessler,17.1,UTA,20.9,1.696552,5.937931,2.7,6.4,8,0.104109
3,Keegan Murray,28.8,SAC,10.2,0.000000,1.892308,4.3,11.9,5,0.101189
4,Jabari Smith Jr.,30.3,HOU,10.2,-0.878571,2.635714,4.0,12.0,14,0.075140


In [7]:
res.to_csv(path + f'Results/roy/results_week_{week}.csv', index = None)